<a href="https://colab.research.google.com/github/JoseTobon/AI_Notebooks/blob/main/Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web screenshot crawler

## Install and import needed libraries

In [14]:
# Install necessary libraries
!pip install selenium
!apt-get update # to update Ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 50 n

## Selenium setup

In [15]:
def setup_driver():
    # Set options for WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Enables headless mode.
    options.add_argument('--no-sandbox')  # Bypass OS security model.
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems.

    # Set the executable path explicitly using Service
    service = Service(ChromeDriverManager().install())

    # Create a Chrome WebDriver instance specifying the options and the service
    driver = webdriver.Chrome(service=service, options=options)
    return driver

## List of websites

In [10]:
urls = [
    "https://www.nytimes.com/es/",
    "https://www.bbc.com/mundo",
    "https://www.nbc.com/?lang=es"
]


## Screenshots

In [16]:
driver = setup_driver()

def take_screenshots_and_save(urls, driver):
    for index, url in enumerate(urls):
        driver.get(url)
        screenshot_path = f"/content/screenshot_{index}.png"
        driver.save_screenshot(screenshot_path)
        print(f"Saved screenshot of {url} as {screenshot_path}")

take_screenshots_and_save(urls, driver)
driver.quit()


AttributeError: 'NoneType' object has no attribute 'split'

## View Screenshots in colab

In [ ]:
from IPython.display import Image, display
for screenshot_file in os.listdir('/content'):
    if screenshot_file.endswith('.png'):
        display(Image(filename=f'/content/{screenshot_file}'))
